In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp38-cp38-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.1
    Uninstalling transformers-4.8.1:
      Successfully uninstalled transformers-4.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from tqdm import tqdm


In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/beaver/offerTrain.xlsx")

In [ ]:
## 1. 중복행 제거
## 2. '사업자등록번호', '상점명', '카테고리명', '상품명' 중복 제거
## 3. 상품명에 ERRO가 포함 안된 것만
## 총 데이터수 360266
data =data[~data['상품명'].str.contains('ERRO')]

In [ ]:
data3

,사업자등록번호,상점명,카테고리명,상품명,표준상품명,가격,label
0,1.464301e+09,BGT단팥빵_불광점,전체메뉴,-3000,-,-3000.0,5609.0
1,1.464301e+09,BGT단팥빵_불광점,전체메뉴,-2000,-,-2000.0,5609.0
2,2.200285e+09,미각상점 강남역지하상가점,마카롱/뚱카롱,0,-,0.0,5609.0
8,4.198601e+09,상황진삼계탕,매장,0,-,0.0,5609.0
10,3.357300e+09,행복한하루,음료수,2%,-,1100.0,5609.0
...,...,...,...,...,...,...,...
428672,1.198185e+09,55갤런(유통),펫샵_사료,ZEAL 뉴질랜드 해산물 리조또 3kg,ZEAL 뉴질랜드 해산물 리조또,36800.0,3205.0
428677,1.198185e+09,55갤런(유통),펫샵_사료,(샘플)zeal사료양고기리조또50g,ZEAL 사료 양고기 리조또,1200.0,14314.0
428678,1.198185e+09,55갤런(유통),펫샵_간식,zeal 사슴힘줄125g,ZEAL 사슴힘줄,10200.0,3175.0
428679,1.198185e+09,55갤런(유통),펫샵_간식,zeal 쇠고기져키125g,ZEAL 쇠고기져키,10200.0,22171.0


In [ ]:
# import pickle

# with open('/content/drive/MyDrive/beaver/train_df.pkl', 'wb') as f:
#   pickle.dump(data, f) 

NameError: ignored

In [ ]:
import pickle
with open('/content/drive/MyDrive/beaver/data_with_label (2).pkl', 'rb') as f:
  data = pickle.load(f)

In [ ]:
import pickle
with open('/content/drive/MyDrive/beaver/data_list.pkl', 'rb') as f:
  data_list = pickle.load(f)

In [ ]:
test = {v:k for k,v in dict(data_list).items()}

In [ ]:
test

{'8503': '종이캐리어',
 '22046': '라면사리오뚜기',
 '3227': '오뚜기라면사리',
 '18178': '오뚜기 사리면',
 '3215': '건가지볶음',
 '12989': '건강하개고구마큐브300G5',
 '18881': '건강하개고구마스틱300G5',
 '189': '건강한 우리집 잡채',
 '11716': '건조과일 슬라이스',
 '13245': '건과일초코 망디앙',
 '11913': '건두부(100g)',
 '13552': '건두부무침',
 '15883': '건두부볶음',
 '4052': '건조레몬',
 '8232': '건망고 50g',
 '10704': '건면',
 '554': '건무화과 큐브 Pack',
 '9857': '연홍도 건미역',
 '10157': '건조빙어',
 '16796': '건새우(국내산)',
 '11287': '건새우달래전',
 '4920': '건새우마늘쫑볶음',
 '6852': '건새우야채전',
 '21214': '건새우볶음',
 '16456': '건생선대',
 '19992': '[승화전사] 건식 용지',
 '4892': '건어물',
 '2321': '건어물소스4종',
 '16131': '(소)건오징어',
 '5491': '수제 다쿠아즈(건자두)',
 '19083': '건전지 AAA',
 '5780': '건채소',
 '3050': '건취나물',
 '5412': '건크랜베리 10g',
 '16576': '스콘(건포도)',
 '9718': '안주(건포도+아몬드)',
 '14769': '건포도 치즈 브레드',
 '3477': '스콘 2종 (건포도호두)',
 '3323': '생생감귤쥬스',
 '4106': '생 고구마 라떼',
 '21361': '생고기김치찌개',
 '22078': '생과일 도시락',
 '1862': '생과일 눈꽃빙수',
 '20697': '생과일 퀸아망',
 '10779': '생과일 와플',
 '6723': '생과일 타르트(라지 사이즈)',
 '12393': '생과일 그으득 화채',
 '18401'

In [ ]:
for key, value in test.items():
  if value == '-':
    print(key)  

In [ ]:
data2 = data.copy()

In [ ]:
data2

,사업자등록번호,상점명,카테고리명,상품명,표준상품명,가격,label
0,1.464301e+09,BGT단팥빵_불광점,전체메뉴,-3000,-,-3000.0,8503.0
1,1.464301e+09,BGT단팥빵_불광점,전체메뉴,-2000,-,-2000.0,8503.0
2,2.200285e+09,미각상점 강남역지하상가점,마카롱/뚱카롱,0,-,0.0,8503.0
8,4.198601e+09,상황진삼계탕,매장,0,-,0.0,8503.0
10,3.357300e+09,행복한하루,음료수,2%,-,1100.0,8503.0
...,...,...,...,...,...,...,...
428672,1.198185e+09,55갤런(유통),펫샵_사료,ZEAL 뉴질랜드 해산물 리조또 3kg,ZEAL 뉴질랜드 해산물 리조또,36800.0,21852.0
428677,1.198185e+09,55갤런(유통),펫샵_사료,(샘플)zeal사료양고기리조또50g,ZEAL 사료 양고기 리조또,1200.0,11755.0
428678,1.198185e+09,55갤런(유통),펫샵_간식,zeal 사슴힘줄125g,ZEAL 사슴힘줄,10200.0,7170.0
428679,1.198185e+09,55갤런(유통),펫샵_간식,zeal 쇠고기져키125g,ZEAL 쇠고기져키,10200.0,21436.0


In [ ]:
data2.drop('표준상품명_label',axis=1)

,사업자등록번호,상점명,카테고리명,상품명,표준상품명,가격
0,1.464301e+09,BGT단팥빵_불광점,전체메뉴,-3000,-,-3000.0
1,1.464301e+09,BGT단팥빵_불광점,전체메뉴,-2000,-,-2000.0
2,2.200285e+09,미각상점 강남역지하상가점,마카롱/뚱카롱,0,-,0.0
8,4.198601e+09,상황진삼계탕,매장,0,-,0.0
10,3.357300e+09,행복한하루,음료수,2%,-,1100.0
...,...,...,...,...,...,...
428672,1.198185e+09,55갤런(유통),펫샵_사료,ZEAL 뉴질랜드 해산물 리조또 3kg,ZEAL 뉴질랜드 해산물 리조또,36800.0
428677,1.198185e+09,55갤런(유통),펫샵_사료,(샘플)zeal사료양고기리조또50g,ZEAL 사료 양고기 리조또,1200.0
428678,1.198185e+09,55갤런(유통),펫샵_간식,zeal 사슴힘줄125g,ZEAL 사슴힘줄,10200.0
428679,1.198185e+09,55갤런(유통),펫샵_간식,zeal 쇠고기져키125g,ZEAL 쇠고기져키,10200.0


In [ ]:
standard_list = list(set(data['표준상품명']))

In [ ]:
len(standard_list)

22801

In [ ]:
for name in tqdm(range(len(standard_list))):
  data2.loc[(data2['표준상품명'] == f"{standard_list[name]}"), 'label'] = name

100%|██████████| 22801/22801 [07:42<00:00, 49.28it/s]


In [ ]:
data2 = data3.copy()

In [ ]:
data2

,사업자등록번호,상점명,카테고리명,상품명,표준상품명,가격,표준상품명_label,label
0,1.464301e+09,BGT단팥빵_불광점,전체메뉴,-3000,-,-3000.0,2188.0,8503.0
1,1.464301e+09,BGT단팥빵_불광점,전체메뉴,-2000,-,-2000.0,2188.0,8503.0
2,2.200285e+09,미각상점 강남역지하상가점,마카롱/뚱카롱,0,-,0.0,2188.0,8503.0
8,4.198601e+09,상황진삼계탕,매장,0,-,0.0,2188.0,8503.0
10,3.357300e+09,행복한하루,음료수,2%,-,1100.0,2188.0,8503.0
...,...,...,...,...,...,...,...,...
428672,1.198185e+09,55갤런(유통),펫샵_사료,ZEAL 뉴질랜드 해산물 리조또 3kg,ZEAL 뉴질랜드 해산물 리조또,36800.0,11219.0,21852.0
428677,1.198185e+09,55갤런(유통),펫샵_사료,(샘플)zeal사료양고기리조또50g,ZEAL 사료 양고기 리조또,1200.0,14631.0,11755.0
428678,1.198185e+09,55갤런(유통),펫샵_간식,zeal 사슴힘줄125g,ZEAL 사슴힘줄,10200.0,1667.0,7170.0
428679,1.198185e+09,55갤런(유통),펫샵_간식,zeal 쇠고기져키125g,ZEAL 쇠고기져키,10200.0,22184.0,21436.0


In [ ]:
data3 = data2.copy()
data3

,사업자등록번호,상점명,카테고리명,상품명,표준상품명,가격,표준상품명_label,label
0,1.464301e+09,BGT단팥빵_불광점,전체메뉴,-3000,-,-3000.0,2188.0,8503.0
1,1.464301e+09,BGT단팥빵_불광점,전체메뉴,-2000,-,-2000.0,2188.0,8503.0
2,2.200285e+09,미각상점 강남역지하상가점,마카롱/뚱카롱,0,-,0.0,2188.0,8503.0
8,4.198601e+09,상황진삼계탕,매장,0,-,0.0,2188.0,8503.0
10,3.357300e+09,행복한하루,음료수,2%,-,1100.0,2188.0,8503.0
...,...,...,...,...,...,...,...,...
428672,1.198185e+09,55갤런(유통),펫샵_사료,ZEAL 뉴질랜드 해산물 리조또 3kg,ZEAL 뉴질랜드 해산물 리조또,36800.0,11219.0,21852.0
428677,1.198185e+09,55갤런(유통),펫샵_사료,(샘플)zeal사료양고기리조또50g,ZEAL 사료 양고기 리조또,1200.0,14631.0,11755.0
428678,1.198185e+09,55갤런(유통),펫샵_간식,zeal 사슴힘줄125g,ZEAL 사슴힘줄,10200.0,1667.0,7170.0
428679,1.198185e+09,55갤런(유통),펫샵_간식,zeal 쇠고기져키125g,ZEAL 쇠고기져키,10200.0,22184.0,21436.0


In [ ]:
data3

,사업자등록번호,상점명,카테고리명,상품명,표준상품명,가격,label
0,1.464301e+09,BGT단팥빵_불광점,전체메뉴,-3000,-,-3000.0,8503.0
1,1.464301e+09,BGT단팥빵_불광점,전체메뉴,-2000,-,-2000.0,8503.0
2,2.200285e+09,미각상점 강남역지하상가점,마카롱/뚱카롱,0,-,0.0,8503.0
8,4.198601e+09,상황진삼계탕,매장,0,-,0.0,8503.0
10,3.357300e+09,행복한하루,음료수,2%,-,1100.0,8503.0
...,...,...,...,...,...,...,...
428672,1.198185e+09,55갤런(유통),펫샵_사료,ZEAL 뉴질랜드 해산물 리조또 3kg,ZEAL 뉴질랜드 해산물 리조또,36800.0,21852.0
428677,1.198185e+09,55갤런(유통),펫샵_사료,(샘플)zeal사료양고기리조또50g,ZEAL 사료 양고기 리조또,1200.0,11755.0
428678,1.198185e+09,55갤런(유통),펫샵_간식,zeal 사슴힘줄125g,ZEAL 사슴힘줄,10200.0,7170.0
428679,1.198185e+09,55갤런(유통),펫샵_간식,zeal 쇠고기져키125g,ZEAL 쇠고기져키,10200.0,21436.0


In [ ]:
data2 = data3.copy()

In [ ]:
data3 = data2.drop('표준상품명_label',axis=1)

In [ ]:
data_list = []
for q, label in tqdm(zip(data2['상품명'], data2['label'])) :
    data2 = []
    data2.append(q)
    data2.append((str(round(label))))

    data_list.append(data2)

360233it [00:00, 360630.92it/s]


In [ ]:
label_list = {}
for q, label in tqdm(zip(data2['표준상품명'], data2['label'])) :
    label_list[q] = label

360233it [00:00, 1878687.26it/s]


In [ ]:
# import pickle

# with open('/content/drive/MyDrive/beaver/data_list.pkl', 'wb') as f:
#   pickle.dump(data_list, f) 

NameError: ignored

In [ ]:
print(data_list[0])
print(data_list[6000])
print(data_list[-1])


['-3000', '8503']
['[에그드랍] 치즈 불고기 길거리 토스트 세트', '8503']
['zeal 양귀125g', '5085']


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 25
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# max_len = 25
# batch_size = 64
# warmup_ratio = 0.1
# num_epochs = 20
# max_grad_norm = 1
# log_interval = 200
# learning_rate =  5e-5

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=22801,   ##클래스 수 조정##
                 dr_rate=0.4,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
import torch, gc
gc.collect()

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
check = torch.load('/content/drive/MyDrive/beaver/checkpoint2.tar')
model.load_state_dict(check["model_state_dict"])
# optimizer.load_state_dict(check["optimizer_state_dict"])

<All keys matched successfully>

In [ ]:
checkpoint=1
for e in tqdm(range(num_epochs)):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))

        if (e + 1) % num_epochs == 0:
          torch.save(
              {
                  "model":model,
                  "epoch": e,
                  "model_state_dict": model.state_dict(),
                  "optimizer_state_dict": optimizer.state_dict(),
                  "description": f"CustomModel 체크포인트-{checkpoint}",
              },
              f"./checkpoint-{checkpoint}.pt",
          )
          checkpoint += 1
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

       


    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-175-1e77d729ae62>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/4222 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 10.258502960205078 train acc 0.0
epoch 1 batch id 201 loss 9.46869945526123 train acc 0.06009017412935323
epoch 1 batch id 401 loss 7.4312424659729 train acc 0.12542861596009974
epoch 1 batch id 601 loss 7.155958652496338 train acc 0.1714070299500832
epoch 1 batch id 801 loss 5.746490001678467 train acc 0.20657771535580524
epoch 1 batch id 1001 loss 5.076600074768066 train acc 0.24221091408591408
epoch 1 batch id 1201 loss 5.172900199890137 train acc 0.27696971273938387
epoch 1 batch id 1401 loss 3.919430732727051 train acc 0.31035867237687365
epoch 1 batch id 1601 loss 3.31809139251709 train acc 0.34120276389756404
epoch 1 batch id 1801 loss 3.053762912750244 train acc 0.3684931982232093
epoch 1 batch id 2001 loss 4.054050445556641 train acc 0.39345952023988007
epoch 1 batch id 2201 loss 2.873905658721924 train acc 0.4162099613811904
epoch 1 batch id 2401 loss 3.4323158264160156 train acc 0.4372136609745939
epoch 1 batch id 2601 loss 2.853341817855835 train acc

<ipython-input-175-1e77d729ae62>:40: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1408 [00:00<?, ?it/s]

...

epoch 2 test acc 0.8490363507231404


  0%|          | 0/4222 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 2.0278191566467285 train acc 0.765625


 67%|██████▋   | 2/3 [1:02:16<31:08, 1868.00s/it]


RuntimeError: ignored

In [ ]:
# 모델 저장
# torch.save({
#         'model_state_dict': model.state_dict(),
#         'optimizer_state_dict': optimizer.state_dict(),
#         # 'epoch' : epoch
#         }, '/content/drive/MyDrive/beaver/checkpoint2.tar')

In [ ]:
check = torch.load('/content/checkpoint-84.pt')
model.load_state_dict(check["model_state_dict"])
optimizer.load_state_dict(check["optimizer_state_dict"])

<All keys matched successfully>

In [ ]:
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:
import pickle
with open('/content/drive/MyDrive/beaver/mpdel.pt', 'wb') as f:
  pickle.dump(model,f)

In [ ]:
# train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=7)
# test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=7)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 7 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
import numpy as np

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            for value, key in label_list.items():
              if np.argmax(logits) == key:
                  test_eval.append(f'{value}')
            # elif np.argmax(logits) == 1:
            #     test_eval.append("놀람이")
            # elif np.argmax(logits) == 2:
            #     test_eval.append("분노가")
            # elif np.argmax(logits) == 3:
            #     test_eval.append("슬픔이")
            # elif np.argmax(logits) == 4:
            #     test_eval.append("중립이")
            # elif np.argmax(logits) == 5:
            #     test_eval.append("행복이")
            # elif np.argmax(logits) == 6:
            #     test_eval.append("혐오가")

        print(test_eval)

In [ ]:
# 표준화된 상품명을 출력해주는 predict값
predict("2%")

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


['흑당소스']
